In [1]:
# Timeseriessplit 0.01 컬럼추가

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy as sp
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')
# Standard plotly imports
#import plotly.plotly as py
import plotly.graph_objs as go
import plotly.tools as tls
from plotly.offline import iplot, init_notebook_mode
#import cufflinks
#import cufflinks as cf
import plotly.figure_factory as ff

# Using plotly + cufflinks in offline mode
init_notebook_mode(connected=True)
#cufflinks.go_offline(connected=True)

# Preprocessing, modelling and evaluating
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold
# from xgboost import XGBClassifier
# import xgboost as xgb

## Hyperopt modules
from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK, STATUS_RUNNING
from functools import partial

import os
import gc
print(os.listdir("../input"))

['test_identity.csv', 'sample_submission.csv', 'train_identity.csv', 'train_transaction.csv', 'test_transaction.csv']


In [3]:
#modeling
df_trans = pd.read_csv('../input/train_transaction.csv', index_col='TransactionID')
df_test_trans = pd.read_csv('../input/test_transaction.csv', index_col='TransactionID')

df_id = pd.read_csv('../input/train_identity.csv', index_col='TransactionID')
df_test_id = pd.read_csv('../input/test_identity.csv', index_col='TransactionID')

sample_submission = pd.read_csv('../input/sample_submission.csv', index_col='TransactionID')

df_train = df_trans.merge(df_id, how='left', left_index=True, right_index=True)
df_test = df_test_trans.merge(df_test_id, how='left', left_index=True, right_index=True)

print(df_train.shape)
print(df_train.shape)

(590540, 433)
(590540, 433)


In [4]:
def resumetable(df):
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values    
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.loc[0].values
    summary['Second Value'] = df.loc[1].values
    summary['Third Value'] = df.loc[2].values

    for name in summary['Name'].value_counts().index:
        summary.loc[summary['Name'] == name, 'Entropy'] = round(stats.entropy(df[name].value_counts(normalize=True), base=2),2) 

    return summary

## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

def CalcOutliers(df_num): 

    # calculating mean and std of the array
    data_mean, data_std = np.mean(df_num), np.std(df_num)

    # seting the cut line to both higher and lower values
    # You can change this value
    cut = data_std * 3

    #Calculating the higher and lower cut values
    lower, upper = data_mean - cut, data_mean + cut

    # creating an array of lower, higher and total outlier values 
    outliers_lower = [x for x in df_num if x < lower]
    outliers_higher = [x for x in df_num if x > upper]
    outliers_total = [x for x in df_num if x < lower or x > upper]

    # array without outlier values
    outliers_removed = [x for x in df_num if x > lower and x < upper]
    
    print('Identified lowest outliers: %d' % len(outliers_lower)) # printing total number of values in lower cut of outliers
    print('Identified upper outliers: %d' % len(outliers_higher)) # printing total number of values in higher cut of outliers
    print('Total outlier observations: %d' % len(outliers_total)) # printing total number of values outliers of both sides
    print('Non-outlier observations: %d' % len(outliers_removed)) # printing total number of non outlier values
    print("Total percentual of Outliers: ", round((len(outliers_total) / len(outliers_removed) )*100, 4)) # Percentual of outliers in points
    
    return

In [5]:
df_train = reduce_mem_usage(df_train)
df_test = reduce_mem_usage(df_test)

Mem. usage decreased to 668.22 Mb (66.2% reduction)
Mem. usage decreased to 583.43 Mb (65.6% reduction)


In [6]:
import datetime
START_DATE = '2017-12-01'
startdate = datetime.datetime.strptime(START_DATE, "%Y-%m-%d")
df_train["Date"] = df_train['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds=x)))
df_train['_Weekdays'] = df_train['Date'].dt.dayofweek
df_train['_Hours'] = df_train['Date'].dt.hour
df_train['_Days'] = df_train['Date'].dt.day

START_DATE_test = '2018-06-01'
startdate = datetime.datetime.strptime(START_DATE_test, "%Y-%m-%d")
df_test["Date"] = df_test['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds=x)))
df_test['_Weekdays'] = df_test['Date'].dt.dayofweek
df_test['_Hours'] = df_test['Date'].dt.hour
df_test['_Days'] = df_test['Date'].dt.day



In [7]:
def corret_card_id(x): 
    x=x.replace('.0','')
    x=x.replace('-999','nan')
    return x

def definie_indexes(df):
    # create card ID 
    cards_cols= ['card1', 'card2', 'card3', 'card5']
    for card in cards_cols: 
        if '1' in card: 
            df['Card_ID']= df[card].map(str)
        else : 
            df['Card_ID']+= ' '+df[card].map(str)
    
    # sort train data by Card_ID and then by transaction date 
    df= df.sort_values(['Card_ID', 'Date'], ascending=[True, True])
    
    # small correction of the Card_ID
    df['Card_ID']=df['Card_ID'].apply(corret_card_id)
    
    # set indexes 
    # df= df.set_index(['Card_ID', 'Date'])
    return df

df_train = definie_indexes(df_train)
df_test = definie_indexes(df_test)


In [8]:
########################### M columns (except M4)
# All these columns are binary encoded 1/0
# We can have some features from it
i_cols = ['M1','M2','M3','M5','M6','M7','M8','M9']

df_train['M_sum'] = df_train[i_cols].sum(axis=1).astype(np.int8)
df_test['M_sum']  = df_test[i_cols].sum(axis=1).astype(np.int8)

df_train['M_na'] = df_train[i_cols].isna().sum(axis=1).astype(np.int8)
df_test['M_na']  = df_test[i_cols].isna().sum(axis=1).astype(np.int8)

df_train['M_type'] = ''
df_test['M_type']  = ''

for col in i_cols:
    df_train['M_type'] = '_'+df_train[col].astype(str)
    df_test['M_type'] = '_'+df_test[col].astype(str)


In [9]:
########################### C columns
# C columns are some counts, based on client identity
# Most popular Value is "1" -> that seems to be just a single match 
# (New or stable client)
# You can check that auc score for that cliens are lower than global
# Lets encode such client types

i_cols = ['C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13','C14']

df_train['C_sum'] = 0
df_test['C_sum']  = 0

df_train['C_null'] = 0
df_test['C_null']  = 0

for col in i_cols:
    df_train['C_sum'] += np.where(df_train[col]==1,1,0)
    df_test['C_sum']  += np.where(df_test[col]==1,1,0)

    df_train['C_null'] += np.where(df_train[col]==0,1,0)
    df_test['C_null']  += np.where(df_test[col]==0,1,0)
    
    valid_values = df_train[col].value_counts()
    valid_values = valid_values[valid_values>1000]
    valid_values = list(valid_values.index)
    
    df_train[col+'_valid'] = np.where(df_train[col].isin(valid_values),1,0)
    df_test[col+'_valid']  = np.where(df_test[col].isin(valid_values),1,0)


In [10]:
# ########################### Reset values for "noise" card1
# valid_card = df_train['card1'].value_counts()
# valid_card = valid_card[valid_card>10]
# valid_card = list(valid_card.index)
    
# df_train['card1'] = np.where(df_train['card1'].isin(valid_card), df_train['card1'], np.nan)
# df_test['card1']  = np.where(df_test['card1'].isin(valid_card), df_test['card1'], np.nan)


In [11]:
########################### Device info
df_train['DeviceInfo'] = df_train['DeviceInfo'].fillna('unknown_device').str.lower()
df_test['DeviceInfo'] = df_test['DeviceInfo'].fillna('unknown_device').str.lower()

df_train['DeviceInfo_c'] = df_train['DeviceInfo']
df_test['DeviceInfo_c'] = df_test['DeviceInfo']

device_match_dict = {
    'sm':'sm-',
    'sm':'samsung',
    'huawei':'huawei',
    'moto':'moto',
    'rv':'rv:',
    'trident':'trident',
    'lg':'lg-',
    'htc':'htc',
    'blade':'blade',
    'windows':'windows',
    'lenovo':'lenovo',
    'linux':'linux',
    'f3':'f3',
    'f5':'f5'
}
for dev_type_s, dev_type_o in device_match_dict.items():
    df_train['DeviceInfo_c'] = df_train['DeviceInfo_c'].apply(lambda x: dev_type_s if dev_type_o in x else x)
    df_test['DeviceInfo_c'] = df_test['DeviceInfo_c'].apply(lambda x: dev_type_s if dev_type_o in x else x)

df_train['DeviceInfo_c'] = df_train['DeviceInfo_c'].apply(lambda x: 'other_d_type' if x not in device_match_dict else x)
df_test['DeviceInfo_c'] = df_test['DeviceInfo_c'].apply(lambda x: 'other_d_type' if x not in device_match_dict else x)


In [12]:
df_train.loc[df_train['R_emaildomain'].isin(['gmail.com', 'gmail']),'R_emaildomain'] = 'Google'

df_train.loc[df_train['R_emaildomain'].isin(['yahoo.com', 'yahoo.com.mx',  'yahoo.co.uk',
                                             'yahoo.co.jp', 'yahoo.de', 'yahoo.fr',
                                             'yahoo.es']), 'R_emaildomain'] = 'Yahoo Mail'
df_train.loc[df_train['R_emaildomain'].isin(['hotmail.com','outlook.com','msn.com', 'live.com.mx', 
                                             'hotmail.es','hotmail.co.uk', 'hotmail.de',
                                             'outlook.es', 'live.com', 'live.fr',
                                             'hotmail.fr']), 'R_emaildomain'] = 'Microsoft'
df_train.loc[df_train.R_emaildomain.isin(df_train.R_emaildomain\
                                         .value_counts()[df_train.R_emaildomain.value_counts() <= 300 ]\
                                         .index), 'R_emaildomain'] = "Others"
df_train.R_emaildomain.fillna("NoInf", inplace=True)


In [13]:
df_test.loc[df_test['R_emaildomain'].isin(['gmail.com', 'gmail']),'R_emaildomain'] = 'Google'

df_test.loc[df_test['R_emaildomain'].isin(['yahoo.com', 'yahoo.com.mx',  'yahoo.co.uk',
                                             'yahoo.co.jp', 'yahoo.de', 'yahoo.fr',
                                             'yahoo.es']), 'R_emaildomain'] = 'Yahoo Mail'
df_test.loc[df_test['R_emaildomain'].isin(['hotmail.com','outlook.com','msn.com', 'live.com.mx', 
                                             'hotmail.es','hotmail.co.uk', 'hotmail.de',
                                             'outlook.es', 'live.com', 'live.fr',
                                             'hotmail.fr']), 'R_emaildomain'] = 'Microsoft'
df_test.loc[df_test.R_emaildomain.isin(df_test.R_emaildomain\
                                         .value_counts()[df_test.R_emaildomain.value_counts() <= 300 ]\
                                         .index), 'R_emaildomain'] = "Others"
df_test.R_emaildomain.fillna("NoInf", inplace=True)


In [14]:
df_train.loc[df_train['id_30'].str.contains('Windows', na=False), 'id_30'] = 'Windows'
df_train.loc[df_train['id_30'].str.contains('iOS', na=False), 'id_30'] = 'iOS'
df_train.loc[df_train['id_30'].str.contains('Mac OS', na=False), 'id_30'] = 'Mac'
df_train.loc[df_train['id_30'].str.contains('Android', na=False), 'id_30'] = 'Android'
df_train['id_30'].fillna("NAN", inplace=True)

In [15]:
df_test.loc[df_test['id_30'].str.contains('Windows', na=False), 'id_30'] = 'Windows'
df_test.loc[df_test['id_30'].str.contains('iOS', na=False), 'id_30'] = 'iOS'
df_test.loc[df_test['id_30'].str.contains('Mac OS', na=False), 'id_30'] = 'Mac'
df_test.loc[df_test['id_30'].str.contains('Android', na=False), 'id_30'] = 'Android'
df_test['id_30'].fillna("NAN", inplace=True)


In [16]:
df_train.loc[df_train['id_31'].str.contains('chrome', na=False), 'id_31'] = 'Chrome'
df_train.loc[df_train['id_31'].str.contains('firefox', na=False), 'id_31'] = 'Firefox'
df_train.loc[df_train['id_31'].str.contains('safari', na=False), 'id_31'] = 'Safari'
df_train.loc[df_train['id_31'].str.contains('edge', na=False), 'id_31'] = 'Edge'
df_train.loc[df_train['id_31'].str.contains('ie', na=False), 'id_31'] = 'IE'
df_train.loc[df_train['id_31'].str.contains('samsung', na=False), 'id_31'] = 'Samsung'
df_train.loc[df_train['id_31'].str.contains('opera', na=False), 'id_31'] = 'Opera'
df_train['id_31'].fillna("NAN", inplace=True)
df_train.loc[df_train.id_31.isin(df_train.id_31.value_counts()[df_train.id_31.value_counts() < 200].index), 'id_31'] = "Others"

In [17]:
df_test.loc[df_test['id_31'].str.contains('chrome', na=False), 'id_31'] = 'Chrome'
df_test.loc[df_test['id_31'].str.contains('firefox', na=False), 'id_31'] = 'Firefox'
df_test.loc[df_test['id_31'].str.contains('safari', na=False), 'id_31'] = 'Safari'
df_test.loc[df_test['id_31'].str.contains('edge', na=False), 'id_31'] = 'Edge'
df_test.loc[df_test['id_31'].str.contains('ie', na=False), 'id_31'] = 'IE'
df_test.loc[df_test['id_31'].str.contains('samsung', na=False), 'id_31'] = 'Samsung'
df_test.loc[df_test['id_31'].str.contains('opera', na=False), 'id_31'] = 'Opera'
df_test['id_31'].fillna("NAN", inplace=True)
df_test.loc[df_test.id_31.isin(df_test.id_31.value_counts()[df_test.id_31.value_counts() < 200].index), 'id_31'] = "Others"


In [18]:
def features_interaction(df, feature_1, feature_2):
    return df[feature_1].astype(str) + '_' + df[feature_2].astype(str)

In [19]:
# Based on https://www.kaggle.com/nroman/lgb-single-model-lb-0-9419

features_interactions = [
    'id_02__id_20',
    'id_02__D8',
    'D11__DeviceInfo',
    'DeviceInfo__P_emaildomain',
    'P_emaildomain__C2',
    'card2__dist1',
    'card1__card5',
    'card2__id_20',
    'card5__P_emaildomain',
    'addr1__card1'
]

for new_feature in features_interactions:
    feature_1, feature_2 = new_feature.split('__')
    
    df_train[new_feature] = features_interaction(df_train, feature_1, feature_2)
    df_test[new_feature] = features_interaction(df_test, feature_1, feature_2)

In [20]:
df_train['count_last'] = df_train.groupby('Card_ID')['TransactionAmt'].transform(lambda x: x.rolling(10, 1).count())
df_train['mean_last'] = df_train.groupby('Card_ID')['TransactionAmt'].transform(lambda x: x.rolling(10, 1).mean())
df_train['min_last'] = df_train.groupby('Card_ID')['TransactionAmt'].transform(lambda x: x.rolling(10, 1).min())
df_train['max_last'] = df_train.groupby('Card_ID')['TransactionAmt'].transform(lambda x: x.rolling(10, 1).max())
df_train['std_last'] = df_train.groupby('Card_ID')['TransactionAmt'].transform(lambda x: x.rolling(10, 1).std())
df_test['count_last'] = df_test.groupby('Card_ID')['TransactionAmt'].transform(lambda x: x.rolling(10, 1).count())
df_test['mean_last'] = df_test.groupby('Card_ID')['TransactionAmt'].transform(lambda x: x.rolling(10, 1).mean())
df_test['min_last'] = df_test.groupby('Card_ID')['TransactionAmt'].transform(lambda x: x.rolling(10, 1).min())
df_test['max_last'] = df_test.groupby('Card_ID')['TransactionAmt'].transform(lambda x: x.rolling(10, 1).max())
df_test['std_last'] = df_test.groupby('Card_ID')['TransactionAmt'].transform(lambda x: x.rolling(10, 1).std())
df_train['trans_mean_last'] = df_train['TransactionAmt'] / df_train.groupby('Card_ID')['TransactionAmt'].transform(lambda x: x.rolling(10, 1).mean())
df_train['trans_std_last'] = df_train['TransactionAmt'] / df_train.groupby('Card_ID')['TransactionAmt'].transform(lambda x: x.rolling(10, 1).std())
df_test['trans_mean_last'] = df_test['TransactionAmt'] / df_test.groupby('Card_ID')['TransactionAmt'].transform(lambda x: x.rolling(10, 1).mean())
df_test['trans_std_last'] = df_test['TransactionAmt'] / df_test.groupby('Card_ID')['TransactionAmt'].transform(lambda x: x.rolling(10, 1).std())
df_train['TransactionAmt_to_mean_card_id'] = df_train['TransactionAmt'] - df_train.groupby(['Card_ID'])['TransactionAmt'].transform('mean')
df_train['TransactionAmt_to_std_card_id'] = df_train['TransactionAmt_to_mean_card_id'] / df_train.groupby(['Card_ID'])['TransactionAmt'].transform('std')
df_test['TransactionAmt_to_mean_card_id'] = df_test['TransactionAmt'] - df_test.groupby(['Card_ID'])['TransactionAmt'].transform('mean')
df_test['TransactionAmt_to_std_card_id'] = df_test['TransactionAmt_to_mean_card_id'] / df_test.groupby(['Card_ID'])['TransactionAmt'].transform('std')
df_train['id_02_to_mean_card_id'] = df_train['id_02'] / df_train.groupby(['Card_ID'])['id_02'].transform('mean')
df_train['id_02_to_std_card_id'] = df_train['id_02'] / df_train.groupby(['Card_ID'])['id_02'].transform('std')
df_test['id_02_to_mean_card_id'] = df_test['id_02'] / df_test.groupby(['Card_ID'])['id_02'].transform('mean')
df_test['id_02_to_std_card_id'] = df_test['id_02'] / df_test.groupby(['Card_ID'])['id_02'].transform('std')
df_train['D15_to_mean_card_id'] = df_train['D15'] / df_train.groupby(['Card_ID'])['D15'].transform('mean')
df_train['D15_to_std_card_id'] = df_train['D15'] / df_train.groupby(['Card_ID'])['D15'].transform('std')
df_test['D15_to_mean_card_id'] = df_test['D15'] / df_test.groupby(['Card_ID'])['D15'].transform('mean')
df_test['D15_to_std_card_id'] = df_test['D15'] / df_test.groupby(['Card_ID'])['D15'].transform('std')
df_train['D15_to_mean_addr1'] = df_train['D15'] / df_train.groupby(['addr1'])['D15'].transform('mean')
df_train['D15_to_std_addr1'] = df_train['D15'] / df_train.groupby(['addr1'])['D15'].transform('std')
df_test['D15_to_mean_addr1'] = df_test['D15'] / df_test.groupby(['addr1'])['D15'].transform('mean')
df_test['D15_to_std_addr1'] = df_test['D15'] / df_test.groupby(['addr1'])['D15'].transform('std')

In [21]:
df_train['card1_count_full'] = df_train['card1'].map(pd.concat([df_train['card1'], df_test['card1']], ignore_index=True).value_counts(dropna=False))
df_test['card1_count_full'] = df_test['card1'].map(pd.concat([df_train['card1'], df_test['card1']], ignore_index=True).value_counts(dropna=False))


df_train['card2_count_full'] = df_train['card2'].map(pd.concat([df_train['card2'], df_test['card2']], ignore_index=True).value_counts(dropna=False))
df_test['card2_count_full'] = df_test['card2'].map(pd.concat([df_train['card2'], df_test['card2']], ignore_index=True).value_counts(dropna=False))

df_train['card3_count_full'] = df_train['card3'].map(pd.concat([df_train['card3'], df_test['card3']], ignore_index=True).value_counts(dropna=False))
df_test['card3_count_full'] = df_test['card3'].map(pd.concat([df_train['card3'], df_test['card3']], ignore_index=True).value_counts(dropna=False))

df_train['card4_count_full'] = df_train['card4'].map(pd.concat([df_train['card4'], df_test['card4']], ignore_index=True).value_counts(dropna=False))
df_test['card4_count_full'] = df_test['card4'].map(pd.concat([df_train['card4'], df_test['card4']], ignore_index=True).value_counts(dropna=False))

df_train['card5_count_full'] = df_train['card5'].map(pd.concat([df_train['card5'], df_test['card5']], ignore_index=True).value_counts(dropna=False))
df_test['card5_count_full'] = df_test['card5'].map(pd.concat([df_train['card5'], df_test['card5']], ignore_index=True).value_counts(dropna=False))

df_train['card6_count_full'] = df_train['card6'].map(pd.concat([df_train['card6'], df_test['card6']], ignore_index=True).value_counts(dropna=False))
df_test['card6_count_full'] = df_test['card6'].map(pd.concat([df_train['card6'], df_test['card6']], ignore_index=True).value_counts(dropna=False))


df_train['addr1_count_full'] = df_train['addr1'].map(pd.concat([df_train['addr1'], df_test['addr1']], ignore_index=True).value_counts(dropna=False))
df_test['addr1_count_full'] = df_test['addr1'].map(pd.concat([df_train['addr1'], df_test['addr1']], ignore_index=True).value_counts(dropna=False))

df_train['addr2_count_full'] = df_train['addr2'].map(pd.concat([df_train['addr2'], df_test['addr2']], ignore_index=True).value_counts(dropna=False))
df_test['addr2_count_full'] = df_test['addr2'].map(pd.concat([df_train['addr2'], df_test['addr2']], ignore_index=True).value_counts(dropna=False))


In [22]:
for feature in ['id_34', 'id_36']:
    # Count encoded for both df_train and df_test
    df_train[feature + '_count_full'] = df_train[feature].map(pd.concat([df_train[feature], df_test[feature]], ignore_index=True).value_counts(dropna=False))
    df_test[feature + '_count_full'] = df_test[feature].map(pd.concat([df_train[feature], df_test[feature]], ignore_index=True).value_counts(dropna=False))
        
for feature in ['id_01', 'id_31', 'id_33', 'id_35', 'id_36']:
    # Count encoded separately for df_train and df_test
    df_train[feature + '_count_dist'] = df_train[feature].map(df_train[feature].value_counts(dropna=False))
    df_test[feature + '_count_dist'] = df_test[feature].map(df_test[feature].value_counts(dropna=False))


In [23]:
df_train['P_isproton']=(df_train['P_emaildomain']=='protonmail.com')
df_train['R_isproton']=(df_train['R_emaildomain']=='protonmail.com')
df_test['P_isproton']=(df_test['P_emaildomain']=='protonmail.com')
df_test['R_isproton']=(df_test['R_emaildomain']=='protonmail.com')


In [24]:
a = np.zeros(df_train.shape[0])
df_train["lasdf_test_browser"] = a
a = np.zeros(df_test.shape[0])
df_test["lasdf_test_browser"] = a
def setbrowser(df):
    df.loc[df["id_31"]=="samsung browser 7.0",'lasdf_test_browser']=1
    df.loc[df["id_31"]=="opera 53.0",'lasdf_test_browser']=1
    df.loc[df["id_31"]=="mobile safari 10.0",'lasdf_test_browser']=1
    df.loc[df["id_31"]=="google search application 49.0",'lasdf_test_browser']=1
    df.loc[df["id_31"]=="firefox 60.0",'lasdf_test_browser']=1
    df.loc[df["id_31"]=="edge 17.0",'lasdf_test_browser']=1
    df.loc[df["id_31"]=="chrome 69.0",'lasdf_test_browser']=1
    df.loc[df["id_31"]=="chrome 67.0 for android",'lasdf_test_browser']=1
    df.loc[df["id_31"]=="chrome 63.0 for android",'lasdf_test_browser']=1
    df.loc[df["id_31"]=="chrome 63.0 for ios",'lasdf_test_browser']=1
    df.loc[df["id_31"]=="chrome 64.0",'lasdf_test_browser']=1
    df.loc[df["id_31"]=="chrome 64.0 for android",'lasdf_test_browser']=1
    df.loc[df["id_31"]=="chrome 64.0 for ios",'lasdf_test_browser']=1
    df.loc[df["id_31"]=="chrome 65.0",'lasdf_test_browser']=1
    df.loc[df["id_31"]=="chrome 65.0 for android",'lasdf_test_browser']=1
    df.loc[df["id_31"]=="chrome 65.0 for ios",'lasdf_test_browser']=1
    df.loc[df["id_31"]=="chrome 66.0",'lasdf_test_browser']=1
    df.loc[df["id_31"]=="chrome 66.0 for android",'lasdf_test_browser']=1
    df.loc[df["id_31"]=="chrome 66.0 for ios",'lasdf_test_browser']=1
    return df
df_train=setbrowser(df_train)
df_test=setbrowser(df_test)


In [25]:
df_train['TransactionAmt_decimal'] = ((df_train['TransactionAmt'] - df_train['TransactionAmt'].astype(int)) * 1000).astype(int)
df_test['TransactionAmt_decimal'] = ((df_test['TransactionAmt'] - df_test['TransactionAmt'].astype(int)) * 1000).astype(int)


In [26]:
########################### Freq encoding
i_cols = ['card1','card2','card3','card5',
          'C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13','C14',
          'D1','D2','D3','D4','D5','D6','D7','D8','D9',
          'addr1','addr2',
          'dist1','dist2',
          'P_emaildomain', 'R_emaildomain',
          'id_01','id_02','id_03','id_04','id_05','id_06','id_07','id_08','id_09','id_10',
          'id_11','id_13','id_14','id_17','id_18','id_19','id_20','id_21','id_22','id_24',
          'id_25','id_26','id_30','id_31','id_32','id_33',#'id_33_0','id_33_1',
          'DeviceInfo','DeviceInfo_c',#'id_30_c','id_30_v','id_31_v',
         ]

for col in i_cols:
    temp_df = pd.concat([df_train[[col]], df_test[[col]]])
    fq_encode = temp_df[col].value_counts().to_dict()   
    df_train[col+'_fq_enc'] = df_train[col].map(fq_encode)
    df_test[col+'_fq_enc']  = df_test[col].map(fq_encode)


In [27]:
one_value_cols = [col for col in df_train.columns if df_train[col].nunique() <= 1]
one_value_cols_test = [col for col in df_test.columns if df_test[col].nunique() <= 1]
one_value_cols == one_value_cols_test



False

In [28]:
many_null_cols = [col for col in df_train.columns if df_train[col].isnull().sum() / df_train.shape[0] > 0.9]
many_null_cols_test = [col for col in df_test.columns if df_test[col].isnull().sum() / df_test.shape[0] > 0.9]

In [29]:
big_top_value_cols = [col for col in df_train.columns if df_train[col].value_counts(dropna=False, normalize=True).values[0] > 0.9]
big_top_value_cols_test = [col for col in df_test.columns if df_test[col].value_counts(dropna=False, normalize=True).values[0] > 0.9]

In [30]:
cols_to_drop = list(set(many_null_cols + many_null_cols_test +
                        big_top_value_cols +
                        big_top_value_cols_test +
                        one_value_cols+ one_value_cols_test))
len(cols_to_drop)
print(len(cols_to_drop),cols_to_drop)

112 ['V116', 'C3_valid', 'C4_valid', 'V102', 'id_23', 'V123', 'V24', 'C3', 'C3_fq_enc', 'V305', 'V25', 'V129', 'V105', 'V298', 'V120', 'id_07', 'id_18_fq_enc', 'V26', 'lasdf_test_browser', 'id_22', 'V137', 'C11_valid', 'C13_valid', 'V65', 'V111', 'V115', 'V136', 'R_isproton', 'V117', 'V301', 'P_isproton', 'V106', 'M_sum', 'C8_valid', 'V297', 'V299', 'C6_valid', 'C2_valid', 'id_26', 'V114', 'V28', 'V109', 'V135', 'V110', 'V121', 'V112', 'V311', 'C7_valid', 'V316', 'V125', 'V281', 'V89', 'V295', 'id_08', 'id_22_fq_enc', 'id_07_fq_enc', 'V86', 'V284', 'V68', 'id_08_fq_enc', 'id_24', 'V119', 'V133', 'V309', 'V55', 'C9_valid', 'V14', 'V88', 'V321', 'V98', 'V107', 'V103', 'dist2_fq_enc', 'D7_fq_enc', 'V77', 'V118', 'V286', 'V134', 'id_25', 'V101', 'id_21_fq_enc', 'V27', 'id_26_fq_enc', 'V318', 'V320', 'V104', 'id_18', 'V132', 'V113', 'V319', 'V67', 'isFraud', 'C12_valid', 'C14_valid', 'dist2', 'C1_valid', 'V122', 'V293', 'V300', 'C5_valid', 'V108', 'V290', 'id_27', 'V23', 'id_21', 'D7', 'id_

In [31]:
cols_to_drop.remove('isFraud')

df_train = df_train.drop(cols_to_drop, axis=1)
df_test = df_test.drop(cols_to_drop, axis=1)

In [32]:
# cat_cols = ['id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29',
#             'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo', 'ProductCD', 'card4', 'card6', 'M4','P_emaildomain',
#             'R_emaildomain', 'card1', 'card2', 'card3',  'card5', 'addr1', 'addr2', 'M1', 'M2', 'M3', 'M5', 'M6', 'M7', 'M8', 'M9', 'Card_ID','id_02__id_20', 'id_02__D8', 
#             'D11__DeviceInfo', 'DeviceInfo__P_emaildomain', 'P_emaildomain__C2', 'card2__dist1', 'card1__card5', 'card2__id_20',
#             'card5__P_emaildomain', 'addr1__card1']

In [33]:
# categorical features encoding
from tqdm import tqdm_notebook
# for col in cat_cols:
#     if col in df_train.columns:
#         le = preprocessing.LabelEncoder()
#         le.fit(list(df_train[col].astype(str).values) + list(df_test[col].astype(str).values))
#         df_train[col] = le.transform(list(df_train[col].astype(str).values))
#         df_test[col] = le.transform(list(df_test[col].astype(str).values))   

for col in tqdm_notebook(df_train.columns):
    if df_train[col].dtype == 'object':
        le = preprocessing.LabelEncoder()
        le.fit(list(df_train[col].astype(str).values) + list(df_test[col].astype(str).values))
        df_train[col] = le.transform(list(df_train[col].astype(str).values))
        df_test[col] = le.transform(list(df_test[col].astype(str).values))

In [34]:
# # 다중공선성 제거

# threshold = 0.98
    
# # Absolute value correlation matrix
# corr_matrix = df_train[df_train['isFraud'].notnull()].corr().abs()

# # Getting the upper triangle of correlations
# upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# # Select columns with correlations above threshold
# to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

# print('There are %d columns to remove.' % (len(to_drop)))
# df_train = df_train.drop(columns = to_drop)
# df_test = df_test.drop(columns = to_drop)

In [35]:
X_train = df_train.drop(['isFraud', 'TransactionDT', 'Date'], axis=1)#df_train.sort_values('TransactionDT').drop(['isFraud', 'TransactionDT', 'Date'], axis=1)
Y_train = df_train['isFraud']#df_train.sort_values('TransactionDT')['isFraud']
X_test = df_test.drop(['TransactionDT','Date'], axis=1)#df_test.sort_values('TransactionDT').drop(['TransactionDT','Date'], axis=1)
del df_train
df_test = df_test[["TransactionDT"]]



In [36]:
X_train = reduce_mem_usage(X_train)
X_test = reduce_mem_usage(X_test)


gc.collect()

Mem. usage decreased to 604.30 Mb (35.2% reduction)
Mem. usage decreased to 529.12 Mb (34.5% reduction)


56

In [37]:
print(X_train.shape)

(590540, 449)


In [38]:
# Hyperparameter 찾기

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from bayes_opt import BayesianOptimization

In [39]:
# scaler = StandardScaler()
# scaler.fit(X_train)
# scaled_x_train = scaler.transform(X_train)

In [40]:
# x_train_train, x_train_valid, y_train_train, y_train_valid = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

# del scaled_x_train

In [41]:
# lgb_train = lgb.Dataset(data=x_train_train.astype('float32'), label=y_train_train.astype('float32'))
# lgb_valid = lgb.Dataset(data=x_train_valid.astype('float32'), label=y_train_valid.astype('float32'))

In [42]:
import os
os.cpu_count()

4

In [43]:
def train_model(learning_rate,num_leaves, min_data_in_leaf, max_depth, bagging_fraction, feature_fraction, lambda_l1, lambda_l2,min_child_weight):
    print("############## New Run ################")
    params = {
        'objective': 'binary',
        'metric': 'auc',
        'is_unbalance': False,
        'boost_from_average': True,
        'num_threads': 4,
        'learning_rate': learning_rate,#0.01, # learning rate
        'num_iterations' : 1000,
        #'n_estimators' : 800,
        'num_leaves': int(num_leaves),
        'min_data_in_leaf': int(min_data_in_leaf),
        'max_depth': int(max_depth),
        'bagging_fraction' : bagging_fraction,
        'feature_fraction' : feature_fraction,
        'lambda_l1': lambda_l1,
        'lambda_l2': lambda_l2,
        'min_child_weight': min_child_weight,
        'bagging_seed' : 11,
        #'early_stopping_round' : 50,
        'verbose' : -1,
        'seed' : 105
    }
    print("PARAMETERS: ")
    print(f"params  = {params}")
    
    tscv =  StratifiedKFold(n_splits=5)#TimeSeriesSplit(n_splits=3)
    sc = []
#     for fold_, (trn_idx, val_idx) in enumerate(folds.split(X, y)):
    for fold_,(train_idx, test_idx) in enumerate(tscv.split(X_train, Y_train)):
#         print( Y_train.iloc[train_idx].head())
        x_train, x_val = X_train.iloc[train_idx,:], X_train.iloc[test_idx,:]
        y_train, y_val = Y_train.iloc[train_idx], Y_train.iloc[test_idx]
        lgb_train = lgb.Dataset(data=x_train.astype('float32'), label=y_train.astype('float32'))
        lgb_valid = lgb.Dataset(data=x_val.astype('float32'), label=y_val.astype('float32'))
    
        lgb_model = lgb.train(params, lgb_train, valid_sets=lgb_valid, verbose_eval=500)
        y = lgb_model.predict(x_train.astype('float32'), num_iteration=lgb_model.best_iteration)
        train_score = roc_auc_score(y_train.astype('float32'), y)        
        y = lgb_model.predict(x_val.astype('float32'), num_iteration=lgb_model.best_iteration)
        score = roc_auc_score(y_val.astype('float32'), y)
        print ("Fold : ", fold_,"train_auc : ",train_score,"val_auc : ", score)
        sc.append(score)
        
    paralst.append(params)
    scorelst.append(np.mean(sc))
    df_para = pd.DataFrame({"parameter" :paralst,"score":scorelst})
    print(df_para)
    df_para.to_csv('para.csv',index=False)

        
    return np.mean(sc)

bounds = {
    'num_leaves': (31, 700),
    'min_data_in_leaf': (0, 200),
    'max_depth': (-1, 70),#(-1, 50), # -> -1
    'learning_rate': (0.001,0.01),#0.006883242363721497,
    'bagging_fraction' : (0.1, 1),
    'feature_fraction' : (0.1, 1),
    'lambda_l1': (0, 1),
    'lambda_l2': (0, 1),
    'min_child_weight': (0.01, 0.1),
}

traintime =True
if traintime :
    paralst,scorelst = [],[]
    bo = BayesianOptimization(train_model, bounds, random_state=105)
    bo.maximize(init_points=3, n_iter=7, acq='ucb', xi=0.0, alpha=1e-6)
    params = {
        'objective': 'binary',
        'metric': 'auc',
        'is_unbalance': False,
        'boost_from_average': True,
        'num_threads': 4,
        'bagging_seed' : 11,
        'learning_rate':bo.max['params']['learning_rate'],#0.01,
        'n_estimators' : 3000,
        'num_leaves': int(bo.max['params']['num_leaves']),
        'min_data_in_leaf': int(bo.max['params']['min_data_in_leaf']),
        'max_depth': int(bo.max['params']['max_depth']),
        'bagging_fraction' : bo.max['params']['bagging_fraction'],
        'feature_fraction' : bo.max['params']['feature_fraction'],
        'lambda_l1': bo.max['params']['lambda_l1'],
        'lambda_l2': bo.max['params']['lambda_l2'],
        'min_child_weight' :  bo.max['params']['min_child_weight'],
       #'early_stopping_round' : 50,
        'verbose' : -1,
        'seed' :105

    }
    print(params)

|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | learni... | max_depth | min_ch... | min_da... | num_le... |
-------------------------------------------------------------------------------------------------------------------------------------
############## New Run ################
PARAMETERS: 
params  = {'objective': 'binary', 'metric': 'auc', 'is_unbalance': False, 'boost_from_average': True, 'num_threads': 4, 'learning_rate': 0.005820760396195786, 'num_iterations': 1000, 'num_leaves': 332, 'min_data_in_leaf': 0, 'max_depth': 3, 'bagging_fraction': 0.17536593150674396, 'feature_fraction': 0.40015566323110086, 'lambda_l1': 0.8593557450886471, 'lambda_l2': 0.10568683531908452, 'min_child_weight': 0.036715112080303196, 'bagging_seed': 11, 'verbose': -1, 'seed': 105}
[500]	valid_0's auc: 0.850092
[1000]	valid_0's auc: 0.854833
Fold :  0 train_auc :  0.8918011992958857 val_auc :  0.8548328230574298
[500]	valid_0's auc: 0.863007
[1000]	valid_0's auc: 0.856532
Fold 

In [44]:
if not traintime :
    params = {
            'objective': 'binary',
            'metric': 'auc',
            'is_unbalance': False,
            'boost_from_average': True,
            'num_threads': 4,
            #'bagging_seed' : 11,
            'learning_rate':0.0070281530255225,
            'n_estimators' : 5000,
            'num_leaves': 326,
            'min_data_in_leaf': 111,
            'max_depth': 47,
            'bagging_fraction' : 0.768240140757717,
            'feature_fraction' : 0.26179858102782727,
            'lambda_l1': 0.1742026705579186,            
            'lambda_l2': 0.4491470796888313,
            'min_child_weight' : 0.008349999182168921,
           #'early_stopping_round' : 50,
            'verbose' : -1,
            'seed' : 40
            }

    print(params)

    # 
    # x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)
    # lgb_train = lgb.Dataset(data=x_train.astype('float32'), label=y_train.astype('float32'))
    # lgb_valid = lgb.Dataset(data=x_val.astype('float32'), label=y_val.astype('float32'))



    # lgb_model = lgb.train(params, lgb_train, valid_sets=lgb_valid, verbose_eval=100,early_stopping_rounds=50)

    # y = lgb_model.predict(X_test.astype('float32'), num_iteration=lgb_model.best_iteration)

    # submission = pd.read_csv('../input/sample_submission.csv', index_col='TransactionID')
    # submission['isFraud'] = y
    # submission.to_csv('submission.csv')


    # Cross validation Score?로 제출?

    tscv =  StratifiedKFold(n_splits=5)#(n_splits=8, shuffle=True, random_state=42)#TimeSeriesSplit(n_splits=5) #? -> 
    sc = []
    #     for fold_, (trn_idx, val_idx) in enumerate(folds.split(X, y)):
    for fold_,(train_idx, test_idx) in enumerate(tscv.split(X_train, Y_train)):
    #         print( Y_train.iloc[train_idx].head())
        x_train, x_val = X_train.iloc[train_idx,:], X_train.iloc[test_idx,:]
        y_train, y_val = Y_train.iloc[train_idx], Y_train.iloc[test_idx]
        lgb_train = lgb.Dataset(data=x_train.astype('float32'), label=y_train.astype('float32'))
        lgb_valid = lgb.Dataset(data=x_val.astype('float32'), label=y_val.astype('float32'))
        lgb_model = lgb.train(params, lgb_train, valid_sets=lgb_valid, verbose_eval=200,early_stopping_rounds=50)
        y = lgb_model.predict(x_train.astype('float32'), num_iteration=lgb_model.best_iteration)
        train_score = roc_auc_score(y_train.astype('float32'), y)        
        y = lgb_model.predict(x_val.astype('float32'), num_iteration=lgb_model.best_iteration)
        score = roc_auc_score(y_val.astype('float32'), y)
        print ("Fold : ", fold_,"train_auc : ",train_score,"val_auc : ", score)
        y = lgb_model.predict(X_test.astype('float32'), num_iteration=lgb_model.best_iteration)
        sc.append(y)

    print(np.mean(sc,axis =0))

    submission = pd.read_csv('../input/sample_submission.csv', index_col='TransactionID')
    submission['isFraud'] = np.mean(sc, axis =0)
    submission.to_csv('submission.csv')

In [45]:
# # single fold로 train하고 그걸로 classifier 만들기

# x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)
# lgb_train = lgb.Dataset(data=x_train.astype('float32'), label=y_train.astype('float32'))
# lgb_valid = lgb.Dataset(data=x_val.astype('float32'), label=y_val.astype('float32'))
# lgb_model = lgb.train(params, lgb_train, valid_sets=lgb_valid, verbose_eval=100,early_stopping_rounds=50)
# clf = lgb.LGBMClassifier(**params, num_boost_round=lgb_model.best_iteration) #데이터 전부다 넣고 학습
# clf.fit(X_train, Y_train)

# submission = pd.read_csv('../input/sample_submission.csv', index_col='TransactionID')
# submission['isFraud'] = clf.predict_proba(X_test)[:, 1]
# submission.to_csv('submission.csv')
# # y = lgb_model.predict(X_test.astype('float32'), num_iteration=lgb_model.best_iteration)


In [46]:
# clf

In [47]:

# clf = lgb.LGBMClassifier(**params, num_boost_round=lgb_model.best_iteration) #데이터 전부다 넣고 학습
# # lgb_train = lgb.Dataset(data=X_train.astype('float32'), label=Y_train.astype('float32'))
# clf.fit(X_train,Y_train)

In [48]:
# submission = pd.read_csv('../input/sample_submission.csv', index_col='TransactionID')
# submission['isFraud'] = clf.predict_proba(X_test)[:, 1]
# submission.to_csv('submission.csv')

In [49]:
# feature_importance_df = pd.concat([
#     pd.Series(X_train.columns),
#     pd.Series(lgb_model.feature_importance())], axis=1)
# feature_importance_df.columns = ['featureName', 'importance']

# # get top 100 features sorted by importance descending
# temp = feature_importance_df.sort_values(by=['importance'], ascending=False).head(100)

# plt.figure(figsize=(20, 20))
# sns.barplot(x="importance", y="featureName", data=temp)
# plt.show()

In [50]:
# %%time

#scaled_x_test = scaler.transform(x_test)

# y = lgb_model.predict(X_test.astype('float32'), num_iteration=lgb_model.best_iteration)

#del scaled_x_test

In [51]:
# submission = pd.read_csv('../input/sample_submission.csv', index_col='TransactionID')
# submission['isFraud'] = y
# submission.to_csv('lightgbm-simple.csv')